[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1arL7bWuF2P3soS3p19MWJeUDtW0Eu5tk?usp=sharing)

# Get Started with Llama 3.1 Models


Llama 3.1 release comes with three sizes of models 7B, 70B and 405B

In this notebook, we will look at :

*  How to access the Llama 3.1 models over a API?
*  Generate Structured Synthetic Instruction Dataset with Llama 3.1 405B


## Setup

Install all the dependencies and import the required python modules.

In [4]:
!pip3 install --upgrade fireworks-ai


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


## Setup your API Key

In order to use the Llama 3.1, you must first obtain Fireworks API Keys. If you don't already have one, you can one by following the instructions [here](https://docs.fireworks.ai/getting-started/quickstart).

In [5]:
from fireworks.client import Fireworks

#replace the FIREWORKS_API_KEY with the key copied in the above step.
client = Fireworks(api_key="FIREWORKS_API_KEY")

## Accessing Llama 3.1 Models using API

We are sending a request to Llama 3.1 405B model, alternatively you can change the model string to access the otherm models.

* accounts/fireworks/models/llama-v3p1-70b-instruct
* accounts/fireworks/models/llama-v3p1-8B-instruct

### Chat Completions API

In [6]:
model_name = "accounts/fireworks/models/llama-v3p1-405b-instruct"

response = client.chat.completions.create(
	model=model_name,
	messages=[{
		"role": "user",
		"content": "Who are you?",
	}],
)
print(response.choices[0].message.content)

I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."


## Generate Synthetic Data



In [7]:
pip install pydantic


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from pydantic import BaseModel, Field

In [9]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum


class Category(str, Enum):
    COUNTRIES = "Countries"
    CAPITALS = "Capitals"
    RIVERS = "Rivers"
    MOUNTAINS = "Mountains"
    LANDMARKS = "Landmarks"
    CLIMATE = "Climate"
    CULTURE = "Culture"

class Difficulty(str, Enum):
    EASY = "Easy"
    MEDIUM = "Medium"
    HARD = "Hard"
    EXPERT = "Expert"

class QuestionType(str, Enum):
    MULTIPLE_CHOICE = "Multiple Choice"
    TRUE_FALSE = "True/False"
    FILL_IN_THE_BLANK = "Fill in the Blank"
    SHORT_ANSWER = "Short Answer"

class Question(BaseModel):
    instruction: str
    context: str
    response: str
    question_type: QuestionType
    category: Category
    difficulty: Difficulty

class GeographyQuizDataset(BaseModel):
    title: str = "World Geography Challenge Dataset"
    description: str = "Dataset for geography quiz questions and answers"
    questions: List[Question]

In [10]:
import json
def generate_question():
    prompt = """Generate a geography quiz question. Format your response as a JSON object with the following structure:
    {
        "instruction": "The full question text",
        "context": "Provide context about the question",
        "response": "The correct answer",
        "question_type": "The type of question (e.g., 'Multiple Choice')",
        "category": "The category should be marked as one of these: Countries, Capitals, Rivers, Mountains, Landmarks, Climate, Culture",
        "difficulty": "The difficulty level of the question (e.g., 'Easy')"
    }"""

    response = client.chat.completions.create(
        model="accounts/fireworks/models/llama-v3p1-405b-instruct",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": "You are a geography expert creating quiz questions."},
            {"role": "user", "content": prompt}
        ]
    )

    question_data = json.loads(response.choices[0].message.content)
    print(question_data)
    return Question(**question_data)

def main(num_questions=10):
    with open("geography_quiz_dataset.jsonl", "w") as f:
        for i in range(num_questions):
            question = generate_question()
            json.dump(question.dict(), f)
            f.write("\n")
            print(f"Generated question {i+1}/{num_questions}: {question.instruction}")

    print(f"Generated and saved {num_questions} questions to geography_quiz_dataset.jsonl")

if __name__ == "__main__":
    main()

{'instruction': 'Which river is the longest in South America and flows through Brazil, Peru, and Colombia before emptying into the Pacific Ocean?', 'context': 'Rivers of the World', 'response': 'Amazon River', 'question_type': 'Multiple Choice', 'category': 'Rivers', 'difficulty': 'Medium'}
Generated question 1/10: Which river is the longest in South America and flows through Brazil, Peru, and Colombia before emptying into the Pacific Ocean?
{'instruction': "What is the world's largest desert, covering about 9,200,000 square kilometers (3,600,000 sq mi), and spanning across several countries in North Africa?", 'context': 'Deserts are large areas of land with very little rainfall and limited vegetation. They can be hot or cold and are found on every continent. The largest hot desert in the world is a significant geographical feature that affects climate, culture, and ecosystems across North Africa.', 'response': 'Sahara', 'question_type': 'Short Answer', 'category': 'Landmarks', 'diffic

## Conclusion

We’re excited to see how the community leverages Llama 3.1 API to create interesting applications.


For more information and to get started with Llama 3.1, visit [docs.fireworks.ai](https://docs.fireworks.ai) or join our [discord community](https://discord.gg/fireworks-ai)